In [ ]:
%matplotlib inline

In [2]:
#import shapefile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
#from mpl_toolkits.basemap import Basemap
from osgeo import gdal
import re

sns.set(color_codes=True)

In [4]:

coco_trees = []
    
data = pd.read_json('../data/tree_coconuts/trees.json')
data = data['features']

coco = re.compile(r'"(\w+)"=>"(\w+)","(\w+)"=>"(\w+)"')
# extract coconut trees
for record in data:
    properties = record['properties']
    geomtery = record['geometry']
    if(properties['other_tags']):
        tags = properties['other_tags'].split(',')
        for tree_tags in tags:
            trees = tree_tags.split('=>')
            if ((trees[0] == '"species"')|(trees[0] =='"genus"')):
                if(re.search('^"Cocos.', trees[1])):
                    # we found coco trees
                    coco_trees.append({"x": record['geometry']['coordinates'][0],\
                                  "y": record['geometry']['coordinates'][1]})

coco_trees = pd.DataFrame(coco_trees)

In [5]:
print("Number of labeled trees:", coco_trees.shape[0])

Number of labeled trees: 6849


In [7]:
sns.jointplot(x="x", y="y", data=coco_trees, kind="kde")

In [ ]:
# Referenced from: https://annefou.github.io/metos_python/04-plotting/
fig = plt.figure(figsize=(15,15))  # a new figure window
ax = fig.add_subplot(1, 1, 1)  # specify (nrows, ncols, axnum)
ax.set_title('Southern Norway and Sweden 29/02/2017  terra 1km', fontsize=14)

# Read the data and metadata
datafile = gdal.Open("../data/aerial_image/kolovai.tif")
bnd1 = datafile.GetRasterBand(1).ReadAsArray()
bnd2 = datafile.GetRasterBand(2).ReadAsArray()
bnd3 = datafile.GetRasterBand(3).ReadAsArray()
nx = datafile.RasterXSize # Raster xsize
ny = datafile.RasterYSize # Raster ysize

img = np.dstack((bnd1, bnd2, bnd3))
gt = datafile.GetGeoTransform()
proj = datafile.GetProjection()

print("Geotransform",gt)
print("proj=", proj)
xres = gt[1]
yres = gt[5]


# get the edge coordinates and add half the resolution 
# to go to center coordinates
xmin = gt[0] + xres * 0.5
xmax = gt[0] + (xres * nx) - xres * 0.5
ymin = gt[3] + (yres * ny) + yres * 0.5
ymax = gt[3] - yres * 0.5
print("xmin=", xmin,"xmax=", xmax,"ymin=",ymin, "ymax=", ymax)

map = Basemap(projection='cyl',llcrnrlat=ymin,urcrnrlat=ymax,\
            llcrnrlon=xmin,urcrnrlon=xmax , resolution='i', ax=ax)

map.imshow(img, origin='upper', ax=ax)
map.drawcountries(color='blue', linewidth=1.5, ax=ax)
map.drawcoastlines(linewidth=1.5, color='red', ax=ax)